<em><p style="font-size:24px">Logistic Regression with "Adult" Dataset (Income > 50.000 Dollars per Year)</p></em>

<u><p style="font-size:20px">1. Formulating the Goal of the Analysis</p></u>

This example focuses on data from the "Adult" dataset. The aim of the analysis is to create a logistic regression model that uses demographic features to determine whether (or why) an individual earns more than 50.000 dollars a year. The model will be tested and used to predict whether or not the yearly income of 3 individuals, with a random set of demographic characteristics, is likely to be higher than 50.000 dollars.

The data is openly available here: http://archive.ics.uci.edu/ml/datasets/Adult

<u><p style="font-size:20px">2. Importing the Relevant Libraries</p></u>

First, we will import the relevant libraries which we need throughout the example:

<em>Additional info</em>: We will activate Seaborn's visuals to overwrite the standard matplotlib styles for plots.

In [424]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
sns.set

<function seaborn.rcmod.set(context='notebook', style='darkgrid', palette='deep', font='sans-serif', font_scale=1, color_codes=True, rc=None)>

<u><p style="font-size:20px">3. Importing and Transforming the Data</p></u>

We will now import (or read) the data, which is provided as a comma delimited .data file:

In [425]:
raw_data = pd.read_csv("Adult_Data_Raw.data")

Moreover, since the import of the .data file will result in columns (and values) that cannot be interpreted by numpy, we will transform the dataset into a numpy friendly format, while also storing it in a Pandas Dataframe:

In [426]:
raw_data_transformed = pd.DataFrame(raw_data.to_numpy())

Lastly, the raw data file has not been provided with column names, which is why we will define them manually by using the "adult.names" info file, which can be downloaded together with the dataset. 

The "adult.names" file identifies the columns as named below:

In [427]:
raw_data_transformed.rename(columns={
    0: "age", 
    1: "workclass",
    2: "fnlwgt",
    3: "education",
    4: "education-num",
    5: "marital-status",
    6: "occupation",
    7: "relationship",
    8: "race",
    9: "sex",
    10: "capital-gain",
    11: "capital-loss",
    12: "hours-per-week",
    13: "native-country",
    14: "income"}, inplace=True)

<u><p style="font-size:20px">4. Exploring the Data</p></u>

We can now take a first look at the data through a summary:

In [428]:
raw_data_transformed.describe(include="all")

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
count,32560,32560,32560,32560,32560,32560,32560,32560,32560,32560,32560,32560,32560,32560,32560
unique,73,9,21647,16,16,7,15,6,5,2,119,92,94,42,2
top,36,Private,203488,HS-grad,9,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,40,United-States,<=50K
freq,898,22696,13,10501,10501,14976,4140,13193,27815,21789,29849,31041,15216,29169,24719


<u><p style="font-size:18px">4.1. Variable Clarification</p></u>

The dataset provides cases of individuals and a variety of demographic features to describe them. Here is a brief description:

<b>age</b>: The age of the respondent. The data is numerical.

<b>workclass</b>: Describes the individuals job position. The data is categorical with the following values: Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked.

<b>fnlwgt</b>: A weight variable, probably to weight the sample to be nationally representative. The data is numerical.

<b>education</b>: Describes the highest educational degree a person has achieved. The data is categorical with the following values: Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc, 9th, 7th-8th, 12th, Masters, 1st-4th, 10th, Doctorate, 5th-6th, Preschool.

<b>education-num</b>: A variable that includes numerical codes for the different types of education.

<b>marital-status</b>: Describes the respondent's marital status. The data is categorical with the following values: Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent, Married-AF-spouse.

<b>occupation</b>: Describes the respondent's occupation. The data is categorical with the following values: Tech-support, Craft-repair, Other-service, Sales, Exec-managerial, Prof-specialty, Handlers-cleaners, Machine-op-inspct, Adm-clerical, Farming-fishing, Transport-moving, Priv-house-serv, Protective-serv, Armed-Forces.

<b>relationship</b>: A variable that includes the relationship status of a person. The data is categorical with the following values: Wife, Own-child, Husband, Not-in-family, Other-relative, Unmarried.

<b>race</b>: Describes the respondent's race. The data is categorical with the following values: White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black.

<b>sex</b>: A person's gender, described as male or female. The data is dichotomous.

<b>capital-gain</b>: Indicates whether a person has realized financial gains from owned capital, and how much. The data is numeric.

<b>capital-loss</b>: Indicates whether a person has suffered a financial loss regarding owned capital, and how much. The data is numeric.

<b>hours-per-week</b>: Describes how many hours per week a respondent works on average. The data is numeric.

<b>native-country</b>: A person's home country. The data is categorical with the values: United-States, Cambodia, England, Puerto-Rico, Canada, Germany, Outlying-US(Guam-USVI-etc), India, Japan, Greece, South, China, Cuba, Iran, Honduras, Philippines, Italy, Poland, Jamaica, Vietnam, Mexico, Portugal, Ireland, France, Dominican-Republic, Laos, Ecuador, Taiwan, Haiti, Columbia, Hungary, Guatemala, Nicaragua, Scotland, Thailand, Yugoslavia, El-Salvador, Trinadad&Tobago, Peru, Hong, Holand-Netherlands.

<b>income</b>: A dichotomous variable that indicates whether or not a person is earning more than 50.000 dollars a year. Describes how many hours per week a respondent works on average. The data is numeric.

<u><p style="font-size:20px">5. Cleaning and Preparing the Data</p></u>

In this section, we will take a closer look at the variables in the dataset with respect to their usefulness for the analysis. We will also identify errors, missing values and/or outliers and clean the dataset in order to prepare it for the analysis.

<u><p style="font-size:18px">5.1. Dropping Unusable Variables:</p></u>

We can see that the dataset has a multitude of variables to describe individuals. However, not all of them are useful for our analysis. We will be dropping a number of them from the dataset and here is why:

1. <b>fnlwgt</b> is a weight variable to be applied in order to make the dataset's results nationally representative. However, the variable carries no importance for our analysis.


2. <b>education-num</b> is a numerical re-code of the categorical variable <b>education</b>. We will at a later point re-code <b>education</b> ourselves into a variable with fewer categories. Therefore, we do not need to keep <b>education-num</b>.


3. <b>relationship</b> does not seem so include any information that is more useful than those already contained in the <b>marital-status</b> variable. Moreover, some of its categories such as "Husband", "Wife" or "Unmarried" overlap with <b>marital-status</b> or are splitting respondents into specific sub-groups that are not relevant for our analysis.


4. There are a few difficulties with <b>capital-gain</b> and <b>capital-loss</b>. First of all, these variables do not add any information to our analysis that is not "obvious" to be begin with, or already expressed in the dependent variable <b>income</b> itself. Capital gains or losses will be very closely linked to a person's overall income, as they can be considered a part of said income. Moreover, both variables have zero values for more than 90% of the sample as we can see below:

In [429]:
gain_zero = raw_data_transformed["capital-gain"].value_counts(1)[:1]
loss_zero = raw_data_transformed["capital-loss"].value_counts(1)[:1]
capital_zero = pd.DataFrame([gain_zero,loss_zero])
capital_zero.rename(columns={0: "% Zero of Sample"}, inplace=True)
capital_zero

,% Zero of Sample
capital-gain,0.916738
capital-loss,0.953348


5. Lastly, we will decide to drop <b>hours-per-week</b> as well as this variable just seems to be another expression of <b>income</b> as well. This is because, for the vast majority of persons in our dataset, hours of work will directly translate into dollars on their account. Meanwhile, we are more interested in the demographic factors that potentially produce a high income, and not the amount of hours put into achieving it.

As a result, we will be dropping <b>fnlwgt</b>, <b>education-num</b>, <b>relationship</b>, <b>capital-gain</b>, <b>capital-loss</b> and <b>hours-per-week</b> from our dataset and save a new copy without them:

In [430]:
data_v001 = raw_data_transformed.drop([
    "fnlwgt", 
    "education-num", 
    "relationship", 
    "capital-gain", 
    "capital-loss",
    "hours-per-week"], axis=1)
data_v001.describe()

,age,workclass,education,marital-status,occupation,race,sex,native-country,income
count,32560,32560,32560,32560,32560,32560,32560,32560,32560
unique,73,9,16,7,15,5,2,42,2
top,36,Private,HS-grad,Married-civ-spouse,Prof-specialty,White,Male,United-States,<=50K
freq,898,22696,10501,14976,4140,27815,21789,29169,24719


<u><p style="font-size:18px">5.2. Transforming Cloumns into the Correct Data Type</p></u>

At this point, the import of the .data file into Jupyter has caused the values in every column of the dataset to be interpreted as objects:

In [431]:
data_v001.dtypes

age               object
workclass         object
education         object
marital-status    object
occupation        object
race              object
sex               object
native-country    object
income            object
dtype: object

The "object" type is not practical for the future operations we plan to perform on our dataset, e.g. categorizations, dummy transformations and the creation of our model. 

To change this, we can use Panda's <em>astype</em> functionality to transform the columns into the format we would expect, and need to work with. We will choose the "integer" format for <b>age</b>, which is numeric, and the "category" format for all other columns, which are caterogical:

In [432]:
data_v001 = data_v001.astype({
"age": "int",
"workclass": "category",
"education": "category",
"marital-status": "category",
"occupation": "category",
"race": "category",
"sex": "category",
"native-country": "category",
"income": "category"})

Let's check whether the transformation has worked correctly:

In [433]:
data_v001.dtypes

age                  int32
workclass         category
education         category
marital-status    category
occupation        category
race              category
sex               category
native-country    category
income            category
dtype: object

<u><p style="font-size:18px">5.3. Identifying and Dealing with Missing Values, Outliers and Errors</p></u>

After we have have transformed our data, we can now identify possible missing values, outliers and/or errors in our dataset. We can start by having a look at the summary table and check each variable:

In [434]:
data_v001.describe(include="all")

,age,workclass,education,marital-status,occupation,race,sex,native-country,income
count,32560.000000,32560,32560,32560,32560,32560,32560,32560,32560
unique,NaN,9,16,7,15,5,2,42,2
top,NaN,Private,HS-grad,Married-civ-spouse,Prof-specialty,White,Male,United-States,<=50K
freq,NaN,22696,10501,14976,4140,27815,21789,29169,24719
mean,38.581634,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,13.640642,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,17.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,28.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,37.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,48.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


<u><p style="font-size:16px">5.3.1. Identifying Missing Values in <b>age</b></p></u>

According to the summary, there are no missing values in the <b>age</b> variable as we have a full base of 32560 respondents, just like in any other columns of the dataset.

To be 100% sure, we can also check the unique values in <b>age</b> and check for missing values with the <em>isnull</em> operator:

In [435]:
np.sort(data_v001["age"].unique())

array([17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
       68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84,
       85, 86, 87, 88, 90])

In [436]:
data_v001["age"].isnull().unique()

array([False])

We can see that the <b>age</b> variable includes values from 17 to 90 years, all cases have data, and there are no missing values present. As a result, there is no need to remove cases from our data with regard to <b>age</b>.

<u><p style="font-size:16px">5.3.2. Identifying Missing Values in <b>workclass</b></p></u>

What about <b>workclass</b>? The summary did not indicate missing values, but there is the possibility that missing values were indicated differently.

In [437]:
np.sort(data_v001["workclass"].unique())

array([' ?', ' Federal-gov', ' Local-gov', ' Never-worked', ' Private',
       ' Self-emp-inc', ' Self-emp-not-inc', ' State-gov', ' Without-pay'],
      dtype=object)

We can observe two things from this result. Firstly, we have a category marked with "?" in the workclass column, indicating missing values and, secondly, every category seems to have an empty space " " preceding it.

The empty space is probably a result of the previous transformations that we applied to our data and thus we might also see this phenomenon appear in the other columns of our dataset. We can test this by checking the unique values of another column with categorical data, such as <b>education</b>:

In [438]:
np.sort(data_v001["education"].unique())

array([' 10th', ' 11th', ' 12th', ' 1st-4th', ' 5th-6th', ' 7th-8th',
       ' 9th', ' Assoc-acdm', ' Assoc-voc', ' Bachelors', ' Doctorate',
       ' HS-grad', ' Masters', ' Preschool', ' Prof-school',
       ' Some-college'], dtype=object)

As expected, the empty space has been added to all categorical values. Since we want to avoid having to write a space whenever we refer to a categorical value, we can remove them from our dataset by using the <em>strip</em> functionality. Since know that the space is always preceding the category at hand, we can appy a <em>left strip</em> to each categrocial column:

In [439]:
data_v001["workclass"] = data_v001["workclass"].str.lstrip()
data_v001["education"] = data_v001["education"].str.lstrip()
data_v001["marital-status"] = data_v001["marital-status"].str.lstrip()
data_v001["occupation"] = data_v001["occupation"].str.lstrip()
data_v001["race"] = data_v001["race"].str.lstrip()
data_v001["sex"] = data_v001["sex"].str.lstrip()
data_v001["native-country"] = data_v001["native-country"].str.lstrip()
data_v001["income"] = data_v001["income"].str.lstrip()

We can test if our approach worked:

In [440]:
np.sort(data_v001["workclass"].unique())

array(['?', 'Federal-gov', 'Local-gov', 'Never-worked', 'Private',
       'Self-emp-inc', 'Self-emp-not-inc', 'State-gov', 'Without-pay'],
      dtype=object)

It seems like we have successfully removed all blank spaces preceding our categorical values. Thus, we can continue with cleaning the data. We concluded that "?" was indicating missing values in the <b>workclass</b> variable. Let us see for how many cases we do not have information on <b>workclass</b>:

In [441]:
data_v001["workclass"].value_counts()

Private             22696
Self-emp-not-inc     2541
Local-gov            2093
?                    1836
State-gov            1297
Self-emp-inc         1116
Federal-gov           960
Without-pay            14
Never-worked            7
Name: workclass, dtype: int64

We can see that 1836 individuals (ca. 6% of the sample) do not have any information provided for their work class. Even though this is a singnificant amount, our dataset would with more than 30.000 cases still be large enough to perform a solid analysis on it. We can therefore decide to drop the cases with missing information.

Moreover, thinking ahead with regard to our categorizations, it would also be wise to remove those individuals who have work without pay as well as those who have never worked. This is because their sub-group is very small and they cannot be grouped together with other positions in a meaningful way.

As a result, we will remove all cases of missing values, as well as persons without pay and those who have never worked:

In [442]:
data_v002 = data_v001[data_v001["workclass"] != "?"]
data_v002 = data_v002[data_v002["workclass"] != "Without-pay"]
data_v002 = data_v002[data_v002["workclass"] != "Never-worked"]
data_v002["workclass"].value_counts()

Private             22696
Self-emp-not-inc     2541
Local-gov            2093
State-gov            1297
Self-emp-inc         1116
Federal-gov           960
Name: workclass, dtype: int64

We have successfully reduced our dataset and can now move on to the next categorical variable <b>education</b>:

<u><p style="font-size:16px">5.3.3. Identifying Missing Values in <b>education</b></p></u>

What about missing values in <b>education</b>?

In [443]:
np.sort(data_v002["education"].unique())

array(['10th', '11th', '12th', '1st-4th', '5th-6th', '7th-8th', '9th',
       'Assoc-acdm', 'Assoc-voc', 'Bachelors', 'Doctorate', 'HS-grad',
       'Masters', 'Preschool', 'Prof-school', 'Some-college'],
      dtype=object)

In [444]:
data_v002["education"].isnull().unique()

array([False])

We can see 16 distinct categories in the <b>education</b> variable, which is what we expected. As a result, there is no need for further adjustments to the sample for now.

<u><p style="font-size:16px">5.3.4. Identifying Missing Values in <b>marital-status</b></p></u>

We will do a similar check for <b>marital-status</b>.

In [445]:
np.sort(data_v002["marital-status"].unique())

array(['Divorced', 'Married-AF-spouse', 'Married-civ-spouse',
       'Married-spouse-absent', 'Never-married', 'Separated', 'Widowed'],
      dtype=object)

In [446]:
data_v002["marital-status"].isnull().unique()

array([False])

Again, no missing values seem to be present. We can proceed.

<u><p style="font-size:16px">5.3.5. Identifying Missing Values in <b>occupation</b></p></u>

<b>Occupation</b> is next on our list:

In [447]:
np.sort(data_v002["occupation"].unique())

array(['Adm-clerical', 'Armed-Forces', 'Craft-repair', 'Exec-managerial',
       'Farming-fishing', 'Handlers-cleaners', 'Machine-op-inspct',
       'Other-service', 'Priv-house-serv', 'Prof-specialty',
       'Protective-serv', 'Sales', 'Tech-support', 'Transport-moving'],
      dtype=object)

In [448]:
data_v002["occupation"].isnull().unique()

array([False])

Once more, no missing values in the <b>occupation</b> variable. We can move on.

<u><p style="font-size:16px">5.3.6. Identifying Missing Values in <b>race</b></p></u>

Next up is <b>Race</b>:

In [449]:
np.sort(data_v002["race"].unique())

array(['Amer-Indian-Eskimo', 'Asian-Pac-Islander', 'Black', 'Other',
       'White'], dtype=object)

In [450]:
data_v002["race"].isnull().unique()

array([False])

We can move on without changes.

<u><p style="font-size:16px">5.3.7. Identifying Missing Values in <b>sex</b></p></u>

What about missing values in <b>sex</b>?:

In [451]:
np.sort(data_v002["sex"].unique())

array(['Female', 'Male'], dtype=object)

In [452]:
data_v002["sex"].isnull().unique()

array([False])

We can move on to the next variable without changes.

<u><p style="font-size:16px">5.3.8. Identifying Missing Values in <b>native-country</b></p></u>

<b>Native-country</b> is the categorical variable with the most categories in our dataset. We might observe missing values here:

In [453]:
np.sort(data_v002["native-country"].unique())

array(['?', 'Cambodia', 'Canada', 'China', 'Columbia', 'Cuba',
       'Dominican-Republic', 'Ecuador', 'El-Salvador', 'England',
       'France', 'Germany', 'Greece', 'Guatemala', 'Haiti',
       'Holand-Netherlands', 'Honduras', 'Hong', 'Hungary', 'India',
       'Iran', 'Ireland', 'Italy', 'Jamaica', 'Japan', 'Laos', 'Mexico',
       'Nicaragua', 'Outlying-US(Guam-USVI-etc)', 'Peru', 'Philippines',
       'Poland', 'Portugal', 'Puerto-Rico', 'Scotland', 'South', 'Taiwan',
       'Thailand', 'Trinadad&Tobago', 'United-States', 'Vietnam',
       'Yugoslavia'], dtype=object)

In [454]:
data_v002["native-country"].isnull().unique()

array([False])

We can see that we have missing values in form of "?" in the native-country variable. Let us check how many cases these apply to:

In [455]:
data_v002["native-country"].value_counts()

United-States                 27490
Mexico                          610
?                               556
Philippines                     187
Germany                         128
Puerto-Rico                     109
Canada                          107
El-Salvador                     100
India                           100
Cuba                             92
England                          86
Jamaica                          80
South                            71
China                            68
Italy                            68
Dominican-Republic               67
Vietnam                          64
Guatemala                        63
Japan                            59
Poland                           56
Columbia                         56
Taiwan                           42
Iran                             42
Haiti                            42
Portugal                         34
Nicaragua                        33
Peru                             30
Greece                      

We can observe two findings from this result. Firstly, we have 556 cases for which no information on the country of origin is available in the dataset. Secondly, United States citizens are heavily overrepresented in the data, in fact so much so, that even if we decided to only create two categories (US Citizen vs. Not US Citizen), our minority category would only make up ca. 10% of the sample. Moreover, summarizing the multitude of other countries in the dataset under just one category called "Other Countries" would not at all reflect the differences between them, e.g. from a cultural perspective. This would make any findings hard to interpret. Consequently, we choose to not investigate the effect of nationality on income given the dataset at hand.

This leads us to the decision to drop the <b>native-country</b> variable from our dataset and continue without it. Moreover, due to this decision we can keep the 556 cases for which no information on the country of origin is available:

In [456]:
data_v003 = data_v002.drop("native-country", axis=1)

<u><p style="font-size:16px">5.3.9. Identifying Missing Values in <b>income</b></p></u>

Last but not least, we will be checking for missing values in our dependent variable <b>income</b>:

In [457]:
np.sort(data_v003["income"].unique())

array(['<=50K', '>50K'], dtype=object)

In [458]:
data_v003["income"].isnull().unique()

array([False])

We can confirm that no missing values are present in our income variable as well. Therefore, we can move on to our categorization:

<u><p style="font-size:18px">5.4. Categorizing the Categorical Data</p></u>

This section deals with the explanation for categorizing our data and the procedure connected to it.

<u><p style="font-size:16px">5.4.1. Explaining the Categories</p></u>

In order to better visualize the decisions for our categorization, we will have another short look on the summary table for our last dataset:

In [459]:
data_v003.describe(include="all")

,age,workclass,education,marital-status,occupation,race,sex,income
count,30703.000000,30703,30703,30703,30703,30703,30703,30703
unique,NaN,6,16,7,14,5,2,2
top,NaN,Private,HS-grad,Married-civ-spouse,Prof-specialty,White,Male,<=50K
freq,NaN,22696,9959,14331,4140,26288,20778,23053
mean,38.439306,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,13.112744,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,17.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,28.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,37.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,47.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN


We can see that every variable apart from <b>sex</b> and <b>income</b> has at least 5 distinct categories. Moreover, while checking for missing values we could see that some categories could be combined in order to make their respective variables easier to work with. For instance, if we were to create a dummy variable for each education level, we would fill our dataset with 15 (16 minus the reference category) dummies for education alone. At the same time, persons with a degree after 10, 11 or 12 years of school would probably not differ so much from each other as to justify an own dummy for each of them. Instead, we can create categories such as "with higher education" and "without higher education", thereby reducing our need for dummies to only 1 (with values 0 or 1) and making <b>education</b> much easier to handle and look at.

In this example, we will also create age intervals for the <b>age</b> variable as these can reflect the different stages of life a person is in and the financial means connected to it, which typically vary throught one's lifetime.

In the following, there will be given a description about the categories that we plan to create for each variable:

<b>"Age"</b> will be coded into 6 groups reflecting different stages in life:

<b>"1"</b> for <b>"17-24"</b><br/>
<b>"2"</b> for <b>"25-34"</b><br/>
<b>"3"</b> for <b>"35-44"</b><br/>
<b>"4"</b> for <b>"45-54"</b><br/>
<b>"5"</b> for <b>"55-64"</b><br/>
<b>"6"</b> for <b>"65+"</b><br/><br/>

<b>"Workplace":</b> will be coded into public and private jobs, with "Private", "Self-emp-not-inc" and "Self-emp-inc" to be coded as private jobs and all government positions as public jobs. The coding will be: 

<b>"0"</b> for <b>"Public"</b><br/>
<b>"1"</b> for <b>"Private"</b><br/><br/>
    
<b>"Education":</b> will be categorized into individuals with and without a higher education. That is, individuals with a higher education have at least some degree of college education or higher. The coding will be:

<b>"0"</b> for <b>"No higher education"</b><br/>
<b>"1"</b> for <b>"Higher education"</b><br/><br/>

<b>"Status":</b> will be coded into persons who are currently in a partnership where they are living with their partner (i.e. Married-AF-spouse and Married-civ-spouse) and those who are not (including due to separation and window(er)s). The coding will be:

<b>"0"</b> for <b>"Not in a relationship"</b><br/>
<b>"1"</b> for <b>"In a relationship"</b><br/><br/>

<b>"Occupation":</b> will be coded into jobs that characterized by manual labour (blue collar) and office jobs that do not require manual labour (white collar). Under this categorization, "Craft-repair", "Farming-fishing", "Handlers-cleaners", "Other-service", "Priv-house-serv", "Protective-serv" and "Transport-moving" were defined as "blue collar" jobs. "Armed Forces" were not added to this category as the title is rather vague but most persons employed by the Armed Forces can be expected to earn a higher salary than most privately employed persons. Therefore, they were added to the "white collar" category. The coding will be: 

<b>"0"</b> for <b>"Blue Collar"</b><br/>
<b>"1"</b> for <b>"White Collar"</b><br/><br/>

<b>"Race":</b> will be categorized as "white" and "non-white" individuals. The coding will be: 

<b>"0"</b> for <b>"Non-white"</b><br/>
<b>"1"</b> for <b>"White"</b><br/><br/>

<b>"Sex":</b> will be coded as:

<b>"0"</b> for <b>"Female"</b><br/>
<b>"1"</b> for <b>"Male"</b><br/><br/>

Lastly, <b>"Income"</b>(our dependent variable) which describes whether or not an individual earn more than 50.000 dollars a year will be coded as:

<b>"0"</b> for <b>Less than or equal to 50.000 Dollars</b><br/> 
<b>"1"</b> for <b>More than 50.000 Dollars</b>

<u><p style="font-size:16px">5.4.2. Creating the Categories</p></u>

We will now create the categories according to the outlined principles. The newly categorized variables will be added as new columns to the dataframe. The <b>Age Groups</b> variable will later on be transformed into 5 dummy variables via Panda's "get_dummy" function.

<u><p style="font-size:14px">5.4.2.1. Creating <b>Age Groups</b></p></u>

To create the <b>Age Groups</b> column, we will be using Panda's <em>cut</em> functionality to define age bins that will place an individual in its respective bin according to the age intervals we defined earlier. The bins are excluding the first value defined for a bin an include the last. By only giving the last accepted values for each age interval (we know our lowest age is 17 and our highest age is 90), and pre-defining the interval labels, we can make the following work:

In [460]:
bins = [16, 24, 34, 44, 54, 64, 90]
labels = ["17-24", "25-34", "35-44", "45-54", "55-64", "65+"]

data_v003["Age Groups"] = pd.cut(data_v003["age"], bins=bins, labels=labels)
pd.DataFrame(data_v003["Age Groups"].unique())

,0
0,45-54
1,35-44
2,25-34
3,17-24
4,55-64
5,65+


We can now go ahead and categorize <b>workclass</b> via <em>mapping</em>:

<u><p style="font-size:14px">5.4.2.2. Creating <b>Workclass</b> coded</p></u>

We can use Panda's <em>replace</em> functionality to create our new categories. As the new categories will be "Private" and "Public", we do not need to change the already existing "Private" category:

In [461]:
data_v003["Workclass_c"] = data_v003["workclass"]

data_v003["Workclass_c"] = data_v003["Workclass_c"].replace(["Self-emp-not-inc", "Self-emp-inc"], "Private")

data_v003["Workclass_c"] = data_v003["Workclass_c"].replace(["Local-gov", "State-gov", "Federal-gov"], "Public")

pd.DataFrame(data_v003["Workclass_c"].unique())

,0
0,Private
1,Public


We can continue with education:

<u><p style="font-size:14px">5.4.2.3. Creating <b>Education</b> coded</p></u>

We will use the same procedure as for <b>workclass</b>

In [462]:
data_v003["Education_c"] = data_v003["education"]

data_v003["Education_c"] = data_v003["Education_c"].replace(
    ["Preschool", "1st-4th", "5th-6th", "7th-8th", "9th", "10th", "11th", "12th", "HS-grad"], "No Higher Education")

data_v003["Education_c"] = data_v003["Education_c"].replace(
    ["Some-college", "Bachelors", "Assoc-voc", "Assoc-acdm", "Masters", "Prof-school", "Doctorate"], "Higher Education")

pd.DataFrame(data_v003["Education_c"].unique())

,0
0,Higher Education
1,No Higher Education


Continuing with marital-status:

<u><p style="font-size:14px">5.4.2.4. Creating <b>Status</b> coded</p></u>

We will use the same well-known procedure:

In [463]:
data_v003["Status_c"] = data_v003["marital-status"]

data_v003["Status_c"] = data_v003["Status_c"].replace(
    ["Divorced", "Married-spouse-absent", "Separated", "Widowed", "Never-married"], "Alone")

data_v003["Status_c"] = data_v003["Status_c"].replace(
    ["Married-AF-spouse", "Married-civ-spouse"], "With Partner")

pd.DataFrame(data_v003["Status_c"].unique())

,0
0,With Partner
1,Alone


Heading over to our second-last variable to be coded: <b>occupation</b>

<u><p style="font-size:14px">5.4.2.5. Creating <b>Occupation</b> coded</p></u>

Once more, we will apply the <em>replace</em> functionality:

In [464]:
data_v003["Occupation_c"] = data_v003["occupation"]

data_v003["Occupation_c"] = data_v003["Occupation_c"].replace(
    ["Adm-clerical", "Armed-Forces", "Exec-managerial", "Prof-specialty", "Sales", "Tech-support"], "White Collar")

data_v003["Occupation_c"] = data_v003["Occupation_c"].replace(
    ["Craft-repair", "Farming-fishing", "Handlers-cleaners", "Machine-op-inspct", "Other-service", 
     "Priv-house-serv", "Protective-serv", "Transport-moving"], "Blue Collar")

pd.DataFrame(data_v003["Occupation_c"].unique())

,0
0,White Collar
1,Blue Collar


Finally, we will recode our last variable: <b>race</b>

<u><p style="font-size:14px">5.4.2.6. Creating <b>race</b> coded</p></u>

We will use the same well-known procedure. As our new categories will be "White" and "Non-White", we do not need to replace the already existing "White" category:

In [465]:
data_v003["Race_c"] = data_v003["race"]

data_v003["Race_c"] = data_v003["Race_c"].replace(
    ["Amer-Indian-Eskimo", "Asian-Pac-Islander", "Black", "Other"], "Non-White")

pd.DataFrame(data_v003["Race_c"].unique())

,0
0,White
1,Non-White


We have now concluded our recoding as their was no need to recode "sex" and "income", which already exist as dichotome variables. 

<u><p style="font-size:16px">5.4.3. Deleting Uncoded Variables and Reordering the Dataset</p></u>

To create our new dataset going forward, we can now drop our "old" uncoded variables. Moreover, we can re-arrange the columns of our dataset to create a better overview. Lastly, we can capitalize the <b>income</b> and <b>sex</b> variables to create consistency in among the columns names. We will view the head of the resulting new dataset:

In [466]:
data_v004 = data_v003.drop(["age","workclass","education","marital-status","occupation","race"], axis=1)

column_names = ["income", "sex", "Age Groups", "Race_c", "Education_c", "Occupation_c", "Workclass_c", "Status_c"]

data_v004 = data_v004.reindex(columns=column_names)

data_v004 = data_v004.rename(columns={"sex": "Sex", "income": "Income"})

data_v004.head()

,Income,Sex,Age Groups,Race_c,Education_c,Occupation_c,Workclass_c,Status_c
0,<=50K,Male,45-54,White,Higher Education,White Collar,Private,With Partner
1,<=50K,Male,35-44,White,No Higher Education,Blue Collar,Private,Alone
2,<=50K,Male,45-54,Non-White,No Higher Education,Blue Collar,Private,With Partner
3,<=50K,Female,25-34,Non-White,Higher Education,White Collar,Private,With Partner
4,<=50K,Female,35-44,White,Higher Education,White Collar,Private,With Partner


<u><p style="font-size:18px">5.5. A Word on Evenly Balanced Dependent Variables</p></u>

An often asked question when performing classification analysis is whether or not the dependent variable should be balanced evenly (e.g. an equal number of persons with an income higher and lower or equal to 50.000 dollars a year).

Several researches have published articles tackling this question and have concluded two things: Firstly, especially for logistic regressions, artificially balancing the dependent variable provides no benefits to the model. Secondly, results will always be better when the whole available sample is used for modelling (e.g. cases should not be removed to decrease the majority group).

See, for instance: "Instance sampling in credit scoring: An empirical study of sample size and balancing" in International Journal of Forecasting 28(1):224–238, March 2012

We will therefore not attempt to balance our dependent variable <b>Income</b> and decide to keep all the sample available to us for our analysis.

<u><p style="font-size:18px">5.6. Creating Numeric Values and Dummies</p></u>

We have now reached the point where we can encode our categorical variables numerically, so we can apply our logistic regression model. While we have created convenient dichotomous variables for all columns but <b>Age Groups</b>, we will choose to code our reference category in each variable as "0" and our comparison category as "1". For <b>Age Groups</b>, we will use Panda's "get_dummies" function to create 5 new dummy variables with the age group 17-24 as reference category:

Our variables and their respective categories will be re-coded as follows:

In [467]:
data_v004["Income"] = data_v004["Income"].replace("<=50K", 0)
data_v004["Income"] = data_v004["Income"].replace(">50K", 1)

data_v004["Sex"] = data_v004["Sex"].replace("Female", 0)
data_v004["Sex"] = data_v004["Sex"].replace("Male", 1)

data_v004["Race_c"] = data_v004["Race_c"].replace("Non-White", 0)
data_v004["Race_c"] = data_v004["Race_c"].replace("White", 1)

data_v004["Education_c"] = data_v004["Education_c"].replace("No Higher Education", 0)
data_v004["Education_c"] = data_v004["Education_c"].replace("Higher Education", 1)

data_v004["Occupation_c"] = data_v004["Occupation_c"].replace("Blue Collar", 0)
data_v004["Occupation_c"] = data_v004["Occupation_c"].replace("White Collar", 1)

data_v004["Workclass_c"] = data_v004["Workclass_c"].replace("Private", 0)
data_v004["Workclass_c"] = data_v004["Workclass_c"].replace("Public", 1)

data_v004["Status_c"] = data_v004["Status_c"].replace("Alone", 0)
data_v004["Status_c"] = data_v004["Status_c"].replace("With Partner", 1)

data_v004.head()

,Income,Sex,Age Groups,Race_c,Education_c,Occupation_c,Workclass_c,Status_c
0,0,1,45-54,1,1,1,0,1
1,0,1,35-44,1,0,0,0,0
2,0,1,45-54,0,0,0,0,1
3,0,0,25-34,0,1,1,0,1
4,0,0,35-44,1,1,1,0,1


In the next step, we will create dummy variables from our <b>Age Groups</b> using Panda's "get_dummies" function. Our reference category (17-24) will be dropped, so we avoid the "dummy variable trap" by introducing multicollinearity (i.e. the 17-24 category could be perfectly predicted by the other age categories):

In [468]:
Age_Dummy = pd.get_dummies(data_v004["Age Groups"], drop_first = True)
Age_Dummy.head()

,25-34,35-44,45-54,55-64,65+
0,0,0,1,0,0
1,0,1,0,0,0
2,0,0,1,0,0
3,1,0,0,0,0
4,0,1,0,0,0


The dummies can now be added to the dataset by using the "join" function:

In [469]:
data_v004 = data_v004.join(Age_Dummy)
data_v004.head()

,Income,Sex,Age Groups,Race_c,Education_c,Occupation_c,Workclass_c,Status_c,25-34,35-44,45-54,55-64,65+
0,0,1,45-54,1,1,1,0,1,0,0,1,0,0
1,0,1,35-44,1,0,0,0,0,0,1,0,0,0
2,0,1,45-54,0,0,0,0,1,0,0,1,0,0
3,0,0,25-34,0,1,1,0,1,1,0,0,0,0
4,0,0,35-44,1,1,1,0,1,0,1,0,0,0


We can now delete our "old" Age Groups variable and quickly re-order our columns for a better overview. We will save the result in a new dataset:

In [470]:
data_v005 = data_v004.drop("Age Groups", axis=1)

column_names_new = ["Income", "Sex", "25-34", "35-44", "45-54", "55-64", "65+",  
                    "Race_c", "Education_c", "Occupation_c", "Workclass_c", "Status_c"]

data_v005 = data_v005.reindex(columns=column_names_new)

data_v005.head()

,Income,Sex,25-34,35-44,45-54,55-64,65+,Race_c,Education_c,Occupation_c,Workclass_c,Status_c
0,0,1,0,0,1,0,0,1,1,1,0,1
1,0,1,0,1,0,0,0,1,0,0,0,0
2,0,1,0,0,1,0,0,0,0,0,0,1
3,0,0,1,0,0,0,0,0,1,1,0,1
4,0,0,0,1,0,0,0,1,1,1,0,1


<u><p style="font-size:18px">5.7. Verifying the Assumptions for Logistic Regression</p></u>

Before we can continue to fit our model, we should have a final check on the necessary preconditions for performing a logistic regression and see if we comply with all of them:

1. We require the dependent variable to be binary -> Fulfilled


2. Our observations should be independent of each other. -> Fulfilled, as we are not looking at time series data or repeated measurements. Our data is cross-sectional data.


3. No multicollinearity among the independent variables. -> We need to test our regressors for multicollinearity before we continue.


4. Logistic regression assumes linearity of independent variables and log odds. -> Fulfilled, since we are only testing dummy variables, and these meet the assumption of linearity by definition.


5. Logistic regression typically requires a large sample size. -> Fulfilled with more than 30.000 cases remaining in our dataset and categorizations that eliminated heavily underpopulated groups.


As a result, the only assumption we need to verify additionally is the independence of our regressors (no multicollinearity). We can check for multicollinearity via the use of <em>Variance Inflation Factors (VIFs)</em>. The idea is that a linear regression is run for each of the regressors, whereas one of them always acts as the dependent, and the others as independent variables. We then receive <em>R² values</em> for each regression, which indicate how well each variable can be explained by others. VIF values of 10 or higher have been described as indicating a problematic amount of collinearity, which should be corrected, possibly by dropping the respective variable from the model.

<u><p style="font-size:16px">5.7.1. Investigating Multicollinearity via <em>Variance Inflation Factors (VIFs)</em></p></u>

We can perform VIF analysis with Statsmodels:

In [471]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

regressors = data_v005.drop(["Income"], axis=1)

vif = pd.DataFrame()
vif["VIF"] = [variance_inflation_factor(regressors.values, i) for i in range(regressors.shape[1])]
vif["Regressors"] = regressors.columns

vif

,VIF,Regressors
0,3.618527,Sex
1,2.118476,25-34
2,2.279711,35-44
3,1.975197,45-54
4,1.492389,55-64
5,1.159934,65+
6,4.604178,Race_c
7,2.672154,Education_c
8,2.661863,Occupation_c
9,1.201603,Workclass_c


Our VIF analysis does not indicate any problematic correlations between our regressors. As a result, we can for now continue with all of them and fit our initial model.

To indicate that we have concluded the data pre-processing stage and our dataset is now ready for analysis, we can save it as our final "v1" version:

In [472]:
data_v1 = data_v005.copy()

<u><p style="font-size:20px">6. Performing the Regression</p></u>

We have arrived at the point where we have prepared our data sufficiently to perform the logistic regression analysis.

However, in order to make sure that we are not overfitting our model on the dataset at hand, we will split our preprocessed dataset into a training and test dataset. The training dataset will be used to fit our model, while the test dataset will be used to objectively test its accuracy.

We will then proceed to fit an initial model to our test dataset and examine whether all of our input variables are providing meaningful contributions to explain our <b>Income</b> variable. Depending on the results, we can then adjust our model by potentially removing unimportant regressors.

We will start by splitting our dataset into train and test data:

<u><p style="font-size:18px">6.1. Splitting the Dataset into Train and Test Data</p></u>

We can apply Sklearn's <em>"train_test_split"</em> method to split our dataset into training and test data. <em>"Train_test_split"</em> provides the advantage that it creates both the input and the target datasets for both train and test data. Moreover, it shuffles the cases in the sets by default, so we can be sure to remove potential effects that may exist due to the order in which cases were saved in the dataset.

A common train-test-split is the 80/20 split, that is 80% of the dataset is chosen as train and 20% as test data. The goal is always to retain the bulk of the data points in the test dataset so we are not losing the underlying patterns in the data due to shrinking our dataset too much. 

We can apply Sklearn's <em>"train_test_split"</em> as follows:

<em>Additional info:</em> We will also set the random shuffle to a specific state (333), so we will attain the same shuffle order each time we run this code again.

In [473]:
targets_raw = data_v1["Income"]
inputs_raw = data_v1.drop("Income", axis=1)

from sklearn.model_selection import train_test_split

inputs_train_initial, inputs_test, targets_train_initial, targets_test = train_test_split(inputs_raw, targets_raw, test_size=0.2, random_state=333) 

We can test if our split has worked correctly:

In [474]:
split_test = pd.DataFrame()
split_test["Dataset"] = ["inputs_train_initial", "inputs_test", "targets_train_initial", "targets_test"]
split_test["Sample Size"] = [len(inputs_train_initial.index), len(inputs_test.index), len(targets_train_initial.index), len(targets_test.index)]
split_test["Sample Percentage"] = split_test["Sample Size"] / len(data_v1.index)
split_test

,Dataset,Sample Size,Sample Percentage
0,inputs_train_initial,24562,0.799987
1,inputs_test,6141,0.200013
2,targets_train_initial,24562,0.799987
3,targets_test,6141,0.200013


Our split worked indeed, we have roughly achieved an 80/20 split for our train and test data. We can now go ahead and fit our initial model.

<u><p style="font-size:18px">6.2. Fitting the Initial Model</p></u>

We can now fit the initial logistic regression with Statsmodels. We will also add an intercept to the model. We should note that we will use a log-logistic model (log-transformation of the outputs) for an easier interpretation of the outcomes:

In [475]:
inputs_train_initial_const = sm.add_constant(inputs_train_initial)
reg_log_initial = sm.Logit(targets_train_initial,inputs_train_initial_const)
results_log_initial = reg_log.fit()

Optimization terminated successfully.
         Current function value: 0.385686
         Iterations 9


Let's look at the summary:

In [476]:
results_log_initial.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                 Income   No. Observations:                24562
Model:                          Logit   Df Residuals:                    24550
Method:                           MLE   Df Model:                           11
Date:                Thu, 30 Apr 2020   Pseudo R-squ.:                  0.3142
Time:                        15:22:44   Log-Likelihood:                -9473.2
converged:                       True   LL-Null:                       -13813.
Covariance Type:            nonrobust   LLR p-value:                     0.000
================================================================================
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
const           -6.7213      0.166    -40.426      0.000      -7.047      -6.395
Sex              0.5366      0.050     10.653      0.000       0.438       0.635
25-34            1.9723      0.156     12.669      0.000       1.667       2.277
35-44            2.6800      0.155     17.338      0.000       2.377       2.983
45-54            2.9888      0.156     19.201      0.000       2.684       3.294
55-64            2.6273      0.160     16.386      0.000       2.313       2.942
65+              2.0474      0.181     11.333      0.000       1.693       2.401
Race_c           0.3257      0.058      5.576      0.000       0.211       0.440
Education_c      0.9605      0.041     23.193      0.000       0.879       1.042
Occupation_c     1.0828      0.042     26.068      0.000       1.001       1.164
Workclass_c      0.1057      0.049      2.142      0.032       0.009       0.202
Status_c         2.0905      0.047     44.845      0.000       1.999       2.182
================================================================================
"""

The p-value of the LLR (Log-Likelyhood-Ration) test tells us that our model is signifcant. It is significantly different from the LL-Null model (a constant-only-model).


McFadden's R² (Pseudo R-aquared) has a value of ca. 0.3. Values between 0.2 and 0.4 are considered a good fit.


Moreover, the p-values for each of our regressors are below 0.05 and thus significant. There is no need to remove any of them.

Since our model is displaying the log(odds), we need to apply the <em>np.exp()</em> (expontential) operator in order to interpret the odds for the coefficients correctly. We can summarize them in a data frame, ordered by strength (or weight) of impact:

In [477]:
coefficients_initial = results_log_initial.params

coefficients_table = pd.DataFrame(np.exp(coefficients_initial), columns=["Weights"])

coefficients_table.sort_values(by=['Weights'], ascending=False)

,Weights
45-54,19.861144
35-44,14.585374
55-64,13.836443
Status_c,8.088659
65+,7.747665
25-34,7.187484
Occupation_c,2.953080
Education_c,2.613130
Sex,1.710193
Race_c,1.384945


<u><p style="font-size:18px">6.3. Interpreting the Results</p></u>

We can see that the "membership" in a specific <b>age group</b> is by far the most defining indicator for an individuals income. As expected, the older and individual, the higher the chances that her/his income would be high. This reflects the effect of seniority in the job with more years on the job market resulting in more senior positions and a higher pay check. The biggest difference can be observed between the reference group (17-24) and the 45-54 year olds. It seems that income peaks in this age group and falls off afterwards, with most persons aged 65+ probably being retired already and thus earning a lower income (however still being almost 8 times more likely to earn more than 50.000 dollars a year comared to the youngsters).

However, we can see that <b>status</b> also has a big impact, meaning that indviduals in active partnerships are about 8 times as likely to earn more than 50.000 dollars a year than those being "alone". This might be due to the fact that the income measured here is the household income and that two persons can contribute more to it than a single person. However, since <b>status</b> in this dataset was only measured by "being married", chances for someone actually being married also increase with age. Therefore, the gaping difference in income between the youngest and the older age groups is probably amplyfied by the fact that there are more married individuals in the older ages. A further indicator for this is that the differences of the odds between the older age groups are not as big with exception of the oldest age group, where retirement plays a role.

As one would expect, individuals with a higher <b>education</b> are almost three times as likely to have a higher income compared to those without. However, the effect between the compared groups is not as pronounced compared with age. However, many persons in the youngest age group might not yet have had the chance to reach their highest possible education. Therefore, education is likely another confounding factor that emphasizes the differences in income with regard to age.

We can observe a similar impact of <b>occupation</b> on a persons income, signalling that "white collar" workers are almost 3 times as likely as "blue collar" workers to have a high income. This is due to the fact that white collar jobs are associated with skilled labour, which is usually rewarded with a higher salary, whereas blue collar positions comprise unskilled labour with lower salaries.

It was also to expect that men are more likely to have a high yearly income than women, which is caused by the pay gap between the <b>genders</b> that still exists in most countries today (and this dataset it from the 1970s). Men are almost twice as likely to earn more than 50.000 dollars a year than women. However, the effect is not as pronounced as it is for many other variables that we tested.

There is also a discrepancy between <b>white and non-white</b> individuals. Being white appears to offer an almost 40% higher chance to receive a high yearly income. Since most of the individuals in the data are from North America, this result mostly reflects the situation there but might be similar in other western countries at least.

Lastly, there is a roughly 11% lower chance to earn more than 50.000 dollars are year for those who work for <b>private companies compared to persons in public positions</b>. Even though the effect is not very big, it might be plausible considering that public institutions mostly offer better working conditions and boni to their workers. Also, seniority is rewarded more linearly in the public sector. Private companies can freely decide which salaries they want to pay and how they want to reward seniority, whereas there are fixed rules for this in public jobs. Moreover, private companies are profit oriented and might apply practices such as employing cheap labour (and dumping) to save money. It can be argued that private companies also have the potential to pay significantly higher salaries than public institutions ever would, but in the broad perspective, there are probably more companies that pay out lower rather than higher salaries compared to what public servants earn.

<b>Conclusion</b>: Married, white men aged 45-54, with a higher education and a white collar job within the public sector have the best chance to have an income above 50.000 dollars yearly, whereas single, non-white women between 17-24, without a higher education and a blue collar job in the private sector have the lowest chances of earning more than 50.000 dollars a year.

None of these results appears to be particularly surprising.

<u><p style="font-size:18px">6.4. Examining the Predictive Power via Confusion Matrix</p></u>

In order to assess the fit of our model, we can create a confusion matrix which can show us how many cases in the training targets were predicted correctly by our model:

Now let's check the accuracy of our model with the confusion matrix:

In [478]:
Confusion = pd.DataFrame(results_log_initial.pred_table(), columns=["Predicted 0", "Predicted 1"], index=["Actual 0","Actual 1"])
Confusion

,Predicted 0,Predicted 1
Actual 0,16749.0,1672.0
Actual 1,2791.0,3350.0


In [479]:
Confusion_Correct = int(Confusion["Predicted 0"][0:1]) + int(Confusion["Predicted 1"][1:2])
Confusion_Total = Confusion.values.sum()
Accuracy = Confusion_Correct / Confusion_Total
Accuracy

0.818296555655077

It seems that our model got ca. 82% of all outcomes in the training dataset classified correctly. This is a pretty good accuracy.

<u><p style="font-size:20px">7. Testing the Model</p></u>

We have now reached the point where we can test our model against our designated test data. To do this, we can add the predictions and the actual results of the test data to the same dataframe. Moreover, we can add a column that rounds up the predicted values and one that subtracts the actual from the predicted values. In all cases where the result is "0", we have a correct prediction. In cases where the result is "1", we have a falsely predicted negative, and in cases where the result is "-1", we have a falsely predicted positive.

We first need to add a constant to our test inputs as well, as our model will expect it:

In [480]:
inputs_test_const = sm.add_constant(inputs_test)

Now we can create new predictions based on our test inputs:

In [481]:
targets_test_predicted = results_log.predict(inputs_test_const)

We summarize the results as described:

In [482]:
predict_table = pd.DataFrame()
predict_table["Actual"] = targets_test
predict_table["Predicted"] = targets_test_predicted
predict_table["Predicted Round"] = np.round(targets_test_predicted)
predict_table["Difference Predicted"] = predict_table["Actual"] - predict_table["Predicted"]
predict_table["Difference Predicted Round"] = predict_table["Actual"] - predict_table["Predicted Round"]
predict_table.head()

,Actual,Predicted,Predicted Round,Difference Predicted,Difference Predicted Round
29965,0,0.021550,0.0,-0.021550,0.0
831,0,0.050879,0.0,-0.050879,0.0
20447,0,0.001666,0.0,-0.001666,0.0
13269,1,0.722099,1.0,0.277901,0.0
16471,0,0.005356,0.0,-0.005356,0.0


We display the correct and incorrect predictions in a data frame:

In [483]:
predict_results_table = pd.DataFrame()
predict_results_table["Predicted as"] = ["Correct","False Negative","False Positive"]
predict_results_table["Counts"] = predict_table["Difference Predicted Round"].value_counts().unique()
predict_results_table

,Predicted as,Counts
0,Correct,5085
1,False Negative,638
2,False Positive,418


We can calculate the accuracy of our model on the test dataset as follows:

In [484]:
Accuracy = predict_results_table["Counts"][0] / sum(predict_results_table["Counts"])
Accuracy

0.8280410356619443

Similar to the accuracy for our training data, we have achieved an accuracy of 83% for our test dataset. This is an increase of around 1% compared to the training dataset. 

On this basis, we can make the claim that, given the inputs we collected from our respondents, we could in ca. 82% - 83% of the time correctly predict whether or not a given individual is earning more than 50.000 dollars a year. Since we have split our dataset before fitting the model, the result for our test data signalizes that we have not overfit our model and it can be used on "unknown" data with at least the same accuracy.

<u><p style="font-size:20px">8. Predicting the Income of 4 Fictional Persons</p></u>

As a last step of this example, we can check for 3 fictional individuals whether, according to our model, these individuals would earn more than 50.000 dollars a year or not.

We are going to test 4 individuals of which 2 are our "stereotypes" that we described earlier when interpreting our results, and 2 others will have a random combination of the characteristics in our dataset:

1. The first person will be a married, white man aged 45-54, with a higher education and a white collar job within the public sector.

2. The second person will be an single, non-white woman aged 17-24, without a higher education and a blue collar job in the private sector.

3. The third person will be a married, white woman aged 25-34, with a higher education and a white collar job in the private sector.

4. The last person will be a divorced, non-white man aged 55-64, without a higher education and a blue collar job in the public sector.

We first need to create our cases in a new dataset. To get the right column names, we can take the column descriptions from our last dataset:

In [485]:
columns = data_v1.drop(["Income"], axis=1)
column_names = columns.columns.values

predict_persons = pd.DataFrame([
    [1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1],
    [1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0]], columns=column_names) 

predict_persons

,Sex,25-34,35-44,45-54,55-64,65+,Race_c,Education_c,Occupation_c,Workclass_c,Status_c
0,1,0,0,1,0,0,1,1,1,1,1
1,0,0,0,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,1,1,1,0,1
3,1,0,0,0,1,0,0,0,0,1,0


In order to receive correct outputs, we again need to add a constant to our new dataset:

In [486]:
predict_persons_const = sm.add_constant(predict_persons)

Now we can predict the outputs according to our model:

In [487]:
predict_persons_income = results_log.predict(predict_persons_const)
predict_persons_income

0    0.797270
1    0.001204
2    0.428154
3    0.030719
dtype: float64

We can see that our stereotype white, older married man has an almost 80% chance to earn more than 50.000 dollars a year, while the chances for an umarried, non-white young woman with the given parameters are next to zero. After the white man, the white married, still young but not very young, highly educated woman has with almost 43% the best chances to have an income over 50.000 dollars. Lastly, our none-white, male, blue collar worker close to his retirement has without a partner only a 3% chance of having a high income.